Now that we're familiar with some of the basic library functionality, we're going to dive into some abstractions that are used for managing the data.  We'll be looking primarily at the `containers` module, which contains a few wrappers and helpers for managing large data sets, in particular large numbers of target pixel files.


In [8]:
from utilities.containers import *

The first thing we'll do is go over the basic data containers within the library.  The first is a wrapper for a fits hdu list in order to save a little memory when dealing with large data sets.

The `BasicFitsContainer` allows us to wrap a fits hdu list to contain more minimal data about an object.  Before we can instantiate it, we need an object to work with.  Going back to our queries and downloads tutorials, we'll grab the data of an object from a local database.

In [4]:
from utilities import data
from utilities.queries import search
from utilities.downloads import urls
from utilities.downloads.downloader import fits_downloader

In [5]:
campaign = 8 # we'll stick to campagin 8 for now

c8_objs = search.search_file(data.FULL_TARGET_LIST, campaign=campaign)

epic_id = c8_objs.keys()[0] # we'll use the first object
print epic_id

220463108


The BasicFitsContainer constructor is a little interesting.  We supply a flag depending on what kind of k2 hlsp we want it to hold ('EV','VJ', or 'PIX' for EVEREST, SFF, and TPF respectively), then we give it the hdu list and fields.  As an example, we'll build a `BasicFitsContainer` from the `EVEREST` processing of our object.

First we'll download our fits file, then we'll create the `BasicFitsContainer`.

In [16]:
everest_url = urls.everest(epic_id, campaign)
everest_hdu = next(fits_downloader([everest_url]))

print everest_hdu.info()

Filename: <type '_io.BytesIO'>
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      29   ()      
  1  ARRAYS        1 BinTableHDU    316   3853R x 13C   [D, D, D, D, J, D, D, D, D, D, D, D, D]   
  2  PIXELS        1 BinTableHDU     19   3853R x 2C   [58D, 333D]   
  3  APERTURE MASK    1 ImageHDU        54   (11, 11)   int64   
  4  POSTAGE STAMPS    1 BinTableHDU     61   11R x 3C   [11D, 11D, 11D]   
  5  HI RES IMAGE    1 ImageHDU        54   (110, 110)   float64   
None


In [17]:
everest_bfc = BasicFitsContainer('EV', everest_hdu, 'FCOR')

Now we'll just go through an inspect some of the attributes of the conatiner.

In [18]:
print everest_bfc.FCOR # the flux column
print everest_bfc.t # the time column
print everest_bfc.EPIC # the EPIC ID
print everest_bfc.processing # the processing contained in the data

[ 305123.77019247  305172.86957408  305154.59823563 ...,  304932.75049315
  304942.14187423  304926.30150723]
[ 2559.06835157  2559.08878378  2559.10921599 ...,  2637.73040318
  2637.7508351   2637.77126692]
220463108
EVEREST


The `BasicFitsContainer` objects are very... well... basic.  The idea is to hold as little info as we need for each object in a neat an accessable way.  Next we'll look at the other two flags, that is 'VJ' and 'PIX'.

First, 'VJ', which is very similar to 'EV'

In [21]:
sff_url = urls.sff(epic_id, campaign)
sff_hdu = next(fits_downloader([sff_url]))

print sff_hdu.info()

sff_bfc = BasicFitsContainer('VJ', sff_hdu, 'FCOR')

print sff_bfc.FCOR # the flux column
print sff_bfc.t # the time column
print sff_bfc.EPIC # the EPIC ID
print sff_bfc.processing # the processing contained in the data

Filename: <type '_io.BytesIO'>
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      25   ()      
  1  BESTAPER      1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  2  CIRC_APER0    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  3  CIRC_APER1    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  4  CIRC_APER2    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  5  CIRC_APER3    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  6  CIRC_APER4    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  7  CIRC_APER5    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  8  CIRC_APER6    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
  9  CIRC_APER7    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
 10  CIRC_APER8    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
 11  CIRC_APER9    1 BinTableHDU    242   3446R x 6C   [D, D, D, D, I, J]   
 12  PRF_

The 'PIX' flag is a little different, it contains target pixel file information, including the relative pixel indices on the Kepler CCD.

In [24]:
tpf_url = urls.tpf(epic_id, campaign)
tpf_hdu = next(fits_downloader([tpf_url]))

print tpf_hdu.info()
print tpf_hdu[1].data.names

tpf_bfc = BasicFitsContainer('PIX', tpf_hdu)

print tpf_bfc.pixels # the flux column, note the different name
print tpf_bfc.col, tpf_bfc.row # the relative pixel coordinates on the CCD
print tpf_bfc.m, tpf_bfc.n # the shape of the postage stamp
print tpf_bfc.EPIC # the EPIC ID

Filename: <type '_io.BytesIO'>
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      54   ()      
  1  TARGETTABLES    1 BinTableHDU    282   3853R x 13C   [D, E, J, 121J, 121E, 121E, 121E, 121E, 121E, J, E, E, 0E]   
  2  APERTURE      1 ImageHDU        48   (11, 11)   int32   
None
['TIME', 'TIMECORR', 'CADENCENO', 'RAW_CNTS', 'FLUX', 'FLUX_ERR', 'FLUX_BKG', 'FLUX_BKG_ERR', 'COSMIC_RAYS', 'QUALITY', 'POS_CORR1', 'POS_CORR2', 'RB_LEVEL']
[[[             nan              nan   8.03227806e+00 ...,  -3.87139954e-02
     2.25768304e+00   7.98819399e+00]
  [             nan   9.57813454e+00   1.56565580e+01 ...,   4.55063200e+00
     4.60757685e+00   8.15919971e+00]
  [  2.64864182e+00   8.97050476e+00   2.21161442e+01 ...,   1.57418575e+01
     1.63996868e+01   8.40962982e+00]
  ..., 
  [  7.52387094e+00   5.51481628e+00   1.36842051e+01 ...,   5.04453049e+01
     4.32998359e-01   7.11088467e+00]
  [  1.58871007e+00   2.80957437e+00   1.505433